In [ ]:
# !pip install emot
# !pip install symspellpy
# !pip install setuptools wheel
# !pip install spacy
# !pip install pyspellchecker
# !pip install spellchecker
# !pip install inflect
# !pip install langdetect
# !pip install google_trans_new
# !pip install deep-translator
# !pip install ekphrasis
# !pip install cdifflib
# !pip install gensim
# !pip install wordninja

In [1]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import re, string, json

from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources
import spacy
from tqdm.notebook import tqdm
from spellchecker import SpellChecker
import inflect

import difflib
import gensim

import pandas as pd

In [2]:
df = pd.read_csv("../data/usa_combined.csv")
df.head()

,Text
0,want find way end racism try force people inje...
1,"this child almost died tetanus, spent months h..."
2,juste se rappeler m. leblanc la pression que l...
3,"you would expect cases rise significantly, cou..."
4,depends whether vaccination status is embedded...


## Used hashtags

In [12]:
text = df['Text']
hashtag_list = []
for tweet in text:
    print(tweet)
    hashtags = re.findall("#[A-Za-z0-9_]+", tweet)
    if len(hashtags) > 0:
        for tag in hashtags:
            hashtag_list.append(tag)

want find way end racism try force people inject themselves. that’s what i see bunch people all races standing up against forced vaccination. not against vaxx.some people need these vaxx &amp; i support those people getting their shot. i draw line at force
this child almost died tetanus, spent months hospital at cost around $1m, his parents still refused vaccination after. this was pre-pandemic.
juste se rappeler m. leblanc la pression que les pm provinciaux et le monde d’affaires le demandait pour pouvoir ré-ouvrir les frontières 😉 le trafic aérien. +les gouv provinciaux, qc inclus vont-ils bloqués les touristes qui n’auront pas de pass-vaccination à l’hôtel au resto
you would expect cases rise significantly, course, vaccination rates come into play as does space. we as adults are not usually confined single space with as many people as school age students. data states that have lifted vs those that have not bears this out.
depends whether vaccination status is embedded qr code itself

TypeError: expected string or bytes-like object

In [13]:
hashtag_list

['#covidvaccineconfidence',
 '#vanguardvaccinationproject',
 '#populationcontrol',
 '#omicron',
 '#vaccination',
 '#mentalhealth',
 '#psychtwitter',
 '#cdnpoli',
 '#onpoli',
 '#covid19',
 '#l',
 '#polqc',
 '#getvaccinated',
 '#vaccineswork',
 '#canada',
 '#superbowlmusicfest',
 '#socal',
 '#superbowlexperience',
 '#sblvi',
 '#hiringphenomenaltalent',
 '#security',
 '#covid19vaccine',
 '#vhcon',
 '#covid19vaccine',
 '#vhcon',
 '#covid19vaccine',
 '#vhcon',
 '#legaultmustresign',
 '#legault',
 '#covid19',
 '#localnews',
 '#newsbreak',
 '#covid',
 '#defundcbc',
 '#defundbbc',
 '#nyc',
 '#godblessamerica',
 '#nonads',
 '#votehelenkicknadlerny10',
 '#dyk',
 '#fightflu',
 '#heartmonth',
 '#nowplaying',
 '#nowspinning',
 '#np',
 '#nowstreaming',
 '#subgenius',
 '#joerogan',
 '#antivaxxers',
 '#nowplaying',
 '#nowspinning',
 '#np',
 '#nowstreaming',
 '#subgenius',
 '#joerogan',
 '#antivaxxers',
 '#covidab',
 '#covid19',
 '#repnotrip',
 '#ableg',
 '#ciuss_mcq',
 '#covid19',
 '#ambulance',
 '#in

## Detecting Languages

In [ ]:
import pycld2 as cld2

tweets = df['Text']
language = []

for each in tweets:
    text_content = each
    _, _, _, detected_language = cld2.detect(text_content, returnVectors=True)
    language.append(detected_language)

In [ ]:
langs = []

for each in language:
    lang_detected = ""
    for i in range(0,len(each)):
        lang_extracted = each[i][2]
        if i != (len(each) - 1):
            lang_detected += lang_extracted + ", "
        else:
            lang_detected += lang_extracted
    langs.append(lang_detected)

In [ ]:
langs

In [ ]:
df['languages'] = langs
df

In [ ]:
df[df['languages'] != 'ENGLISH'].head()

## Language Translation

In [ ]:
from deep_translator import GoogleTranslator

non_english = df[df['languages'] != 'ENGLISH']

translated = []

for each in non_english:
    translate = GoogleTranslator(source='auto', target='en').translate(each)
    translated.append(translate)

In [ ]:
translated

In [ ]:
#creating new_text column
df['new_text'] = df['Text']

In [ ]:
non_english = df[df['languages'] != 'ENGLISH']

In [ ]:
non_english.head()

In [ ]:
indexes_list = non_english.index

In [ ]:
indexes_list

In [ ]:
len(indexes_list)

In [ ]:
range_firstNum = start_range + start_batch

new_indexes_list = []

for num in indexes_list:
    new = num - range_firstNum
    new_indexes_list.append(new)

In [ ]:
for i in range(0,len(indexes_list)):
    index = new_indexes_list[i]
    df.iloc[index]['new_text'] = translated[i]

In [ ]:
#check that the new_text col to see to check that the initial non-english rows have been translated
df[df['languages'] != 'ENGLISH']

## Language Detection Part 2

In [ ]:
import pycld2 as cld2

reviews = df['new_text']
language2 = []

for each in reviews:
    text_content = each
    _, _, _, detected_language = cld2.detect(text_content, returnVectors=True)
    language2.append(detected_language)

In [ ]:
langs2 = []

for each in language2:
    lang_detected = ""
    for i in range(0,len(each)):
        lang_extracted = each[i][2]
        if i != (len(each) - 1):
            lang_detected += lang_extracted + ", "
        else:
            lang_detected += lang_extracted
    langs2.append(lang_detected)

In [ ]:
langs2

In [ ]:
df['new_languages'] = langs2

In [ ]:
df

In [ ]:
non_english2 = df[df['new_languages'] != 'ENGLISH']
non_english2

In [ ]:
#see how many still detected as non-english, brieflcheck is it rly non english
non_english2.info()

In [ ]:
# non_english2.to_csv('non_english2.csv')

## Emoji Processing

In [3]:
import pickle
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO

In [4]:
# # 'Emoji_Dict.p'- download link https://drive.google.com/open?id=1G1vIkkbqPBYPKHcQ8qy0G2zkoab2Qv4v
with open('./Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# # Function for converting emojis into word
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

In [10]:
# TEST: line of text before emoji processing
df["Text"][32]

'💩trudeau 💩wants 100% vaccination 💉**new video*** via'

In [11]:
# TEST: convert_emoji function
convert_emojis("🍫 🤩")

'chocolate_bar star-struck'

In [ ]:
# emoji processing on df["text"] column

df["new_text"] = df["new_text"].apply(convert_emojis)
df["new_text"][32]

## Emoticon Processing

In [22]:
# 'Emoticon_Dict.p'- download link https://drive.google.com/open?id=1HDpafp97gCl9xZTQWMgP2kKK_NuhENlE
with open('./Emoticon_Dict.p', 'rb') as fp:
    Emoticon_Dict = pickle.load(fp)
    
# Function for converting emoticons into word
def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = text.replace(emot, "_".join(EMOTICONS_EMO[emot].replace(",","").replace(":","").split()))
    return text

In [23]:
# TEST: convert_emoticons function
sentence_listtext = "Hello :-) :-)"
convert_emoticons(sentence_listtext)

'Hello Happy_face_smiley Happy_face_smiley'

In [ ]:
df["new_text"] = df["new_text"].apply(convert_emoticons)

## Text File & Corpus Creation

In [ ]:
# create txt file for each text review

# def CreateCorpusFromDataFrame(corpusfolder, df):
#     for index, r in df.iterrows():
#     # print (r['text'])
#     # print (str(index) + '.txt')
#         text = r['new_text']
#         fname = str(index)+'.txt'
#         corpusfile = open(corpusfolder+'/'+fname, "w", encoding="utf-8")
#         corpusfile.write(str(text))
#         corpusfile.close()

# CreateCorpusFromDataFrame('../text-corpus-usa', df)

# reference: https://stackoverflow.com/questions/49088978/how-to-create-corpus-from-pandas-data-frame-to-operate-with-nltk/49104725

In [ ]:
file_directory = '../text-corpus-usa'
filename_pattern = '.+\.txt'

my_corpus = PlaintextCorpusReader(file_directory, filename_pattern)

#Print some words in specific file
test = my_corpus.words('0.txt')
print(test)
print(len(test))

In [ ]:
#Total number of text files i.e. total number of reviews
print(len(my_corpus.fileids()))

In [ ]:
fdist = nltk.FreqDist(test)
print(fdist.most_common(10))

In [ ]:
# word distribution across all files
fdist_all_files = nltk.FreqDist(my_corpus.words())
print(fdist_all_files.most_common(10))

In [ ]:
#total number of words
total_words = my_corpus.words()
print(total_words)
print(len(total_words))

## Tokenization

In [ ]:
#Creating a sorted list of texts since my_corpus.fileids() does not read texts consecutively
#EDIT RANGE
sorted_list = [f'{i}.txt' for i in range(0, 81041)]

In [ ]:
tokenised_words = [my_corpus.words(each) for each in sorted_list]
tokenised_words

In [ ]:
print(len(tokenised_words))